In [1]:
from main import RFest
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import time

In [2]:
#Checking number of cores
estimator=RFest(drop_rate=0.99999)

In [3]:
estimator.rows_range

[100, 1000, 10000]

In [27]:
df=estimator.generate_data()

2018-09-01 16:35:01,766 - main - Generating dummy training durations to create a training set


In [7]:
estimator.model_fit()

2018-09-01 17:35:30,751 - main - Generating dummy training durations to create a training set
2018-09-01 17:35:34,315 - main - Fitting LR to estimate training durations
2018-09-01 17:35:34,330 - main - Saving LR coefs in json file
2018-09-01 17:35:34,333 - main - Saving LR to LR_estimator.pkl
2018-09-01 17:35:34,340 - main - R squared on train set is 1.0
2018-09-01 17:35:34,342 - main - MAPE on train set is: 1.5967033308610918e-13
2018-09-01 17:35:34,347 - main - MAPE on test set is: 34.59030194142593
2018-09-01 17:35:34,349 - main - MSE on train set is 1.6490775148815707e-32
2018-09-01 17:35:34,352 - main - MSE on test set is 0.0009548975861206668


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [3]:
rf= RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=5, max_leaf_nodes=5, min_impurity_decrease=0.0,
           min_impurity_split=5, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)


In [4]:
estimator.estimate_duration(np.random.rand(20000000,10),rf)


2018-09-01 18:25:48,861 - main - Loading LR coefs from json file
2018-09-01 18:25:48,878 - main - Training your model should take ~ 25.5884222205 seconds


   num_rows  num_features  n_estimators  max_depth  min_samples_split  \
0  20000000            10           100         10                  2   

   min_samples_leaf  min_weight_fraction_leaf  max_features_auto  \
0                 1                       0.0                  0   

   max_leaf_nodes  min_impurity_decrease  min_impurity_split  bootstrap  \
0               5                    0.0                   5       True   

   oob_score  n_jobs  
0      False       8  


25.588422220518776

In [5]:
X,y=np.random.rand(20000000,10),np.random.rand(20000000,1)
start_time = time.time()
rf.fit(X,y)
elapsed_time = time.time() - start_time

In [6]:
elapsed_time

46.80437779426575